In [85]:
%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
%autoreload 2

In [87]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
from pylab import plot, show, savefig
from scipy import stats
#%pylab inline

In [88]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-03-25/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
files_start = 1350
files_end = 1351


In [89]:
# cup configuration
#cup_config = cycle_Sb
cup_config = cycles2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "126"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125"}
# Mass Range of cup configuration
mass_range = cycles2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124"], ["117", "118", "119", "122", "124"]]
isotopes = [["112", "114", "115", "116", "117", "118", "119", "122", "124"]]
#isotopes = [["116", "117", "118", "119", "122", "124"]]
denom_isotope = "120"
isotope_column = ["112", "114", "115", "116", "117", "118", "119", "122", "124"]
data_sample_column = [(i + "/" + "120") for i in isotope_column]
print data_sample_column
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = False

blk_corr = False

['112/120', '114/120', '115/120', '116/120', '117/120', '118/120', '119/120', '122/120', '124/120']


In [90]:
columns_1 = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)"]
#columns_2 = ["cycle", "H8 (2)", "H7 (2)", "H6 (2)", "H5 (2)", "H4 (2)", "H3 (2)", "H2 (2)", "H1 (2)", "Ax (2)", "L1 (2)", "L2 (2)", "L3 (2)", "L4 (2)"]
df_all = pd.DataFrame()
blk_ls = []
sample_ls = []
files_1 = range(files_start, files_end, 1)
for sample in files_1:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero_1 = pd.DataFrame(df_zero["cycle1"])
    df_zero_2 = pd.DataFrame(df_zero["cycle2"])
    
df_zero_1

#for x in [123]:
#    sample_ls.remove(x)
#blk_ls

,Ax (1),H1 (1),H2 (1),H3 (1),H4 (1),H5 (1),H6 (1),H7 (1),H8 (1),L1 (1),L2 (1),L3 (1),L4 (1)
1,0.266194,0.138508,6.042085,3.232436,10.347184,3.719947,14.312355,2.088744,2.679955,0.000050,0.381922,0.000117,0.000125
2,0.275136,0.143157,6.244880,3.340780,10.694237,3.844664,14.792802,2.158839,2.769872,0.000047,0.394721,0.000110,0.000118
3,0.275080,0.143119,6.244518,3.340829,10.693736,3.844239,14.791600,2.158527,2.769553,0.000056,0.394775,0.000120,0.000114
4,0.268878,0.139913,6.102966,3.264624,10.450367,3.757051,14.454166,2.109382,2.706433,0.000067,0.385813,0.000140,0.000137
5,0.261040,0.135808,5.925621,3.169998,10.147721,3.648217,14.035733,2.048337,2.628168,0.000094,0.374629,0.000130,0.000141
6,0.273225,0.142206,6.201933,3.317743,10.620373,3.818153,14.689731,2.143882,2.750552,0.000096,0.392042,0.000118,0.000115
7,0.272317,0.141712,6.181454,3.306906,10.585737,3.805734,14.642119,2.136878,2.741669,0.000072,0.390765,0.000140,0.000173
8,0.259029,0.134718,5.876730,3.143642,10.062509,3.617528,13.917712,2.031112,2.605883,0.000068,0.371555,0.000143,0.000143
9,0.247270,0.128706,5.612723,3.002614,9.610934,3.455083,13.292619,1.939775,2.488361,0.000091,0.354896,0.000134,0.000137
10,0.254206,0.132306,5.794125,3.087403,9.883302,3.567235,13.724927,2.003029,2.570035,0.000059,0.364800,0.000127,0.000128


In [223]:
Cd = 0.03
Cd_trans = 320.3
U = 0.00
U_trans = 292.1
U_prod = 0.05
Te = 0
Te_trans = 252.1

In [225]:
Cd_111 = Cd_trans * (Cd/1000.0)*(12.722/100.0)
Cd_112 = Cd_trans * (Cd/1000.0)*(24.211/100.0)
Cd_114 = Cd_trans * (Cd/1000.0)*(28.419/100.0)
Cd_116 = Cd_trans * (Cd/1000.0)*(7.298/100.0)
In_115 = 0.0000
U_double = U_trans * U_prod * (U/1000.0)*(99.2745/100.0)
Te_120 = Te_trans * (Te/1000.0)*(0.0927/100.0)
Te_122 = Te_trans * (Te/1000.0)*(2.5277/100.0)
Te_124 = Te_trans * (Te/1000.0)*(4.7165/100.0)
Te_125 = Te_trans * (Te/1000.0)*(7.0509/100.0)
Te_126 = Te_trans * (Te/1000.0)*(18.8066/100.0)

print Te_124
print df_zero_1_sim["L2 (1)"].mean()/Cd_112
print df_zero_1_sim["Ax (1)"].mean()/Cd_114
#print df_zero_1_sim["H7 (1)"].mean()/Te_122
#print df_zero_1_sim["H8 (1)"].mean()/Te_124
#print df_zero_1_sim["H5 (1)"].mean()/U_double
#Te_125 = 0

0.0
183.242319866
105.751336898


In [226]:
Sn120_sig = 14.30
Sn112 = 0.9718
Sn114 = 0.6583
Sn115 = 0.3379
Sn116 = 14.5385
Sn117 = 7.6706
Sn118 = 24.2178
Sn119 = 8.5871
Sn120 = 32.5974
Sn122 = 4.6316
Sn124 = 5.7889

df_zero_1_sim = pd.DataFrame(columns = df_zero_1.columns, index = range(1, 21))
df_zero_1_sim["L4 (1)"] = np.random.normal(0, 0.00005, 20)
df_zero_1_sim["L3 (1)"] = np.random.normal(0, 0.00005, 20)
df_zero_1_sim["L2 (1)"] = np.random.normal(Sn120_sig*(Sn112/Sn120), 0.00005, 20)
df_zero_1_sim["L1 (1)"] = np.random.normal(0, 0.00005, 20)
df_zero_1_sim["Ax (1)"] = np.random.normal(Sn120_sig*(Sn114/Sn120), 0.00005, 20)
df_zero_1_sim["H1 (1)"] = np.random.normal(Sn120_sig*(Sn115/Sn120), 0.00005, 20)
df_zero_1_sim["H2 (1)"] = np.random.normal(Sn120_sig*(Sn116/Sn120), 0.00005, 20)
df_zero_1_sim["H3 (1)"] = np.random.normal(Sn120_sig*(Sn117/Sn120), 0.00005, 20)
df_zero_1_sim["H4 (1)"] = np.random.normal(Sn120_sig*(Sn118/Sn120), 0.00005, 20)
df_zero_1_sim["H5 (1)"] = np.random.normal(Sn120_sig*(Sn119/Sn120), 0.00005, 20)
df_zero_1_sim["H6 (1)"] = np.random.normal(Sn120_sig*(Sn120/Sn120), 0.00005, 20)
df_zero_1_sim["H7 (1)"] = np.random.normal(Sn120_sig*(Sn122/Sn120), 0.00005, 20)
df_zero_1_sim["H8 (1)"] = np.random.normal(Sn120_sig*(Sn124/Sn120), 0.00005, 20)

df_zero_2_sim = pd.DataFrame(columns = df_zero_2.columns, index = range(1, 21))
df_zero_2_sim["L4 (2)"] = np.random.normal(Sn120_sig*(Sn117/Sn120), 0.00005, 20)
df_zero_2_sim["L3 (2)"] = np.random.normal(Sn120_sig*(Sn118/Sn120), 0.00005, 20)
df_zero_2_sim["L2 (2)"] = np.random.normal(Sn120_sig*(Sn119/Sn120), 0.00005, 20)
df_zero_2_sim["L1 (2)"] = np.random.normal(Sn120_sig*(Sn120/Sn120), 0.00005, 20)
df_zero_2_sim["Ax (2)"] = np.random.normal(0, 0.00005, 20)
df_zero_2_sim["H1 (2)"] = np.random.normal(Sn120_sig*(Sn122/Sn120), 0.00005, 20)
df_zero_2_sim["H2 (2)"] = np.random.normal(0, 0.00005, 20)
df_zero_2_sim["H3 (2)"] = np.random.normal(Sn120_sig*(Sn124/Sn120), 0.00005, 20)
df_zero_2_sim["H4 (2)"] = np.random.normal(0, 0.00005, 20)
df_zero_2_sim["H5 (2)"] = np.random.normal(0, 0.00005, 20)
df_zero_2_sim["H6 (2)"] = np.random.normal(0, 0.00005, 20)
df_zero_2_sim["H7 (2)"] = np.random.normal(0, 0.00005, 20)
df_zero_2_sim["H8 (2)"] = np.random.normal(0, 0.00005, 20)
df_zero_sim = {"cycle1" : df_zero_1_sim.to_dict(), "cycle2" : df_zero_2_sim.to_dict()}

In [227]:
df_zero_1_new = df_zero_1.copy()
df_zero_2_new = df_zero_2.copy()
df_zero_1_new["L3 (1)"] = df_zero_1["L3 (1)"] + Cd_111
df_zero_1_new["L2 (1)"] = df_zero_1["L2 (1)"] + Cd_112
df_zero_1_new["Ax (1)"] = df_zero_1["Ax (1)"] + Cd_114
df_zero_1_new["H2 (1)"] = df_zero_1["H2 (1)"] + Cd_116
df_zero_1_new["H1 (1)"] = df_zero_1["H1 (1)"] + In_115
df_zero_1_new["H5 (1)"] = df_zero_1["H5 (1)"] + U_double
df_zero_1_new["H6 (1)"] = df_zero_1["H6 (1)"] + Te_120
df_zero_1_new["H7 (1)"] = df_zero_1["H7 (1)"] + Te_122
df_zero_1_new["H8 (1)"] = df_zero_1["H8 (1)"] + Te_124

df_zero_2_new["L2 (2)"] = df_zero_2["L2 (2)"] + U_double
df_zero_2_new["L1 (2)"] = df_zero_2["L1 (2)"] + Te_120
df_zero_2_new["H1 (2)"] = df_zero_2["H1 (2)"] + Te_122
df_zero_2_new["H3 (2)"] = df_zero_2["H3 (2)"] + Te_124
df_zero_2_new["H4 (2)"] = df_zero_2["H4 (2)"] + Te_125
df_zero_2_new["H5 (2)"] = df_zero_2["H5 (2)"] + Te_126

df_zero_new = {"cycle1" : df_zero_1_new.to_dict(), "cycle2" : df_zero_2_new.to_dict()}

df_zero_1_sim_int = df_zero_1_sim.copy()
df_zero_2_sim_int = df_zero_2_sim.copy()
df_zero_1_sim_int["L3 (1)"] = df_zero_1_sim["L3 (1)"] + Cd_111
df_zero_1_sim_int["L2 (1)"] = df_zero_1_sim["L2 (1)"] + Cd_112
df_zero_1_sim_int["Ax (1)"] = df_zero_1_sim["Ax (1)"] + Cd_114
df_zero_1_sim_int["H2 (1)"] = df_zero_1_sim["H2 (1)"] + Cd_116
df_zero_1_sim_int["H1 (1)"] = df_zero_1_sim["H1 (1)"] + In_115
df_zero_1_sim_int["H5 (1)"] = df_zero_1_sim["H5 (1)"] + U_double
df_zero_1_sim_int["H6 (1)"] = df_zero_1_sim["H6 (1)"] + Te_120
df_zero_1_sim_int["H7 (1)"] = df_zero_1_sim["H7 (1)"] + Te_122
df_zero_1_sim_int["H8 (1)"] = df_zero_1_sim["H8 (1)"] + Te_124

df_zero_2_sim_int["L2 (2)"] = df_zero_2_sim["L2 (2)"] + U_double
df_zero_2_sim_int["L1 (2)"] = df_zero_2_sim["L1 (2)"] + Te_120
df_zero_2_sim_int["H1 (2)"] = df_zero_2_sim["H1 (2)"] + Te_122
df_zero_2_sim_int["H3 (2)"] = df_zero_2_sim["H3 (2)"] + Te_124
df_zero_2_sim_int["H4 (2)"] = df_zero_2_sim["H4 (2)"] + Te_125
df_zero_2_sim_int["H5 (2)"] = df_zero_2_sim["H5 (2)"] + Te_126

df_zero_sim_int = {"cycle1" : df_zero_1_sim_int.to_dict(), "cycle2" : df_zero_2_sim_int.to_dict()}

In [232]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_internal_norm = pd.DataFrame()
counter = 0 

for dataframe in [df_zero_sim_int , df_zero_sim]:
    #[df_zero_sim_int , df_zero_sim]
    #[df_zero_new , df_zero]
    df = NU_data_read(path, 1350, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)

    new_corr = evaluation(dataframe, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
    new_corr.line2_corr(dataframe, "119")

    data_sample_raw = new_corr.raw_signals(denom_isotope)
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = data_sample_raw[np.abs(data_sample_raw-data_sample_raw.mean())<=(2*data_sample_raw.std())]

    data_sample = new_corr.mass_fractionation(denom_isotope, 10)
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_sample = data_sample[np.abs(data_sample-data_sample.mean())<=(2*data_sample.std())]

    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_sample = data_sample.drop(denom_isotope, axis=1)
    data_sample = data_sample[sorted(data_sample.columns)]
    data_sample.columns = data_sample_column
    data_sample["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample["sample"] = sample_name
    data_sample["cycle"] = data_sample.index.values
    data_sample["counter"] = counter

    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_sample = data_sample.merge(data_sample_raw)

    df_internal_norm = df_internal_norm.append(data_sample)
    counter += 1
df_internal_norm

,112/120,114/120,115/120,116/120,117/120,118/120,119/120,122/120,124/120,date,...,120,122,115,114,117,116,124,112,119,118
0,0.029976,0.020387,0.010364,0.446046,0.235311,0.742939,0.263433,0.142085,0.177590,2015-03-25 14:39:00,...,14.299946,2.031796,0.148211,0.291545,3.364971,6.378521,2.539496,0.428664,3.767091,10.624062
1,0.029975,0.020384,0.010367,0.446052,0.235315,0.742944,0.263428,0.142091,0.177583,2015-03-25 14:39:00,...,14.299981,2.031901,0.148250,0.291490,3.364987,6.378500,2.539447,0.428639,3.767006,10.624060
2,0.029975,0.020384,NaN,0.446047,0.235309,0.742937,0.263431,0.142080,0.177589,2015-03-25 14:39:00,...,14.299971,2.031743,NaN,0.291498,3.364926,6.378482,2.539505,0.428639,3.767054,10.623998
3,0.029976,0.020383,0.010368,0.446047,0.235310,0.742929,0.263431,0.142087,0.177591,2015-03-25 14:39:00,...,14.300023,2.031844,0.148265,0.291486,3.364943,6.378502,2.539548,0.428654,3.767067,10.623920
4,0.029974,0.020384,0.010363,0.446049,0.235317,0.742943,0.263425,0.142087,0.177592,2015-03-25 14:39:00,...,14.300008,2.031862,0.148191,0.291484,3.364983,NaN,2.539615,0.428606,3.766955,10.623997
5,0.029971,0.020386,0.010363,0.446046,0.235313,0.742933,0.263424,0.142085,0.177592,2015-03-25 14:39:00,...,14.299915,2.031779,0.148200,0.291524,3.365012,6.378567,2.539491,0.428608,3.766963,10.623996
6,0.029980,0.020389,0.010363,0.446050,0.235309,0.742935,0.263430,0.142085,0.177585,2015-03-25 14:39:00,...,14.299965,2.031806,0.148188,0.291567,3.364923,6.378538,2.539446,0.428713,3.767051,10.623968
7,0.029973,0.020389,0.010368,0.446051,0.235316,0.742938,0.263429,0.142087,0.177589,2015-03-25 14:39:00,...,14.300012,2.031844,0.148267,0.291559,3.365016,6.378515,2.539531,0.428607,3.767032,10.624001
8,0.029973,0.020385,0.010367,0.446049,0.235315,0.742933,0.263432,0.142082,0.177590,2015-03-25 14:39:00,...,14.299931,2.031759,0.148256,0.291503,3.365011,6.378523,2.539502,0.428620,3.767071,10.623934
9,0.029973,0.020386,0.010366,0.446050,0.235314,0.742937,0.263434,0.142082,0.177591,2015-03-25 14:39:00,...,14.300007,2.031779,0.148229,0.291525,3.364998,6.378518,2.539554,0.428614,3.767112,10.624005


In [233]:
df_internal_norm= df_internal_norm.set_index("counter")
columns = ["cycle", "sample"]
columns.extend(data_sample_column)
columns.extend(sorted(data_sample_raw.drop("cycle", axis=1).columns))
df_internal_sub = pd.DataFrame(df_internal_norm, columns = columns)
df_internal_sub
#df_external_sub.to_csv(path_sav + "Sn_H9_L4_internal_corr_all.csv")

,cycle,sample,112/120,114/120,115/120,116/120,117/120,118/120,119/120,122/120,...,112,114,115,116,117,118,119,120,122,124
counter,,,,,,,,,,,,,,,,,,,,,
0,1,SPEX 100ppb,0.029976,0.020387,0.010364,0.446046,0.235311,0.742939,0.263433,0.142085,...,0.428664,0.291545,0.148211,6.378521,3.364971,10.624062,3.767091,14.299946,2.031796,2.539496
0,2,SPEX 100ppb,0.029975,0.020384,0.010367,0.446052,0.235315,0.742944,0.263428,0.142091,...,0.428639,0.291490,0.148250,6.378500,3.364987,10.624060,3.767006,14.299981,2.031901,2.539447
0,3,SPEX 100ppb,0.029975,0.020384,NaN,0.446047,0.235309,0.742937,0.263431,0.142080,...,0.428639,0.291498,NaN,6.378482,3.364926,10.623998,3.767054,14.299971,2.031743,2.539505
0,4,SPEX 100ppb,0.029976,0.020383,0.010368,0.446047,0.235310,0.742929,0.263431,0.142087,...,0.428654,0.291486,0.148265,6.378502,3.364943,10.623920,3.767067,14.300023,2.031844,2.539548
0,5,SPEX 100ppb,0.029974,0.020384,0.010363,0.446049,0.235317,0.742943,0.263425,0.142087,...,0.428606,0.291484,0.148191,NaN,3.364983,10.623997,3.766955,14.300008,2.031862,2.539615
0,6,SPEX 100ppb,0.029971,0.020386,0.010363,0.446046,0.235313,0.742933,0.263424,0.142085,...,0.428608,0.291524,0.148200,6.378567,3.365012,10.623996,3.766963,14.299915,2.031779,2.539491
0,7,SPEX 100ppb,0.029980,0.020389,0.010363,0.446050,0.235309,0.742935,0.263430,0.142085,...,0.428713,0.291567,0.148188,6.378538,3.364923,10.623968,3.767051,14.299965,2.031806,2.539446
0,8,SPEX 100ppb,0.029973,0.020389,0.010368,0.446051,0.235316,0.742938,0.263429,0.142087,...,0.428607,0.291559,0.148267,6.378515,3.365016,10.624001,3.767032,14.300012,2.031844,2.539531
0,9,SPEX 100ppb,0.029973,0.020385,0.010367,0.446049,0.235315,0.742933,0.263432,0.142082,...,0.428620,0.291503,0.148256,6.378523,3.365011,10.623934,3.767071,14.299931,2.031759,2.539502


In [234]:
def SE(x):
    SE = np.std(x) / x.count()
    return SE

df_internal_norm_mean = df_internal_sub.drop("cycle", axis=1).groupby(df_internal_sub.index).mean()
df_internal_norm_mean_sd = df_internal_sub.drop("cycle", axis=1).groupby(df_internal_sub.index).agg([np.mean, SE])
df_internal_norm_mean["sample"] = df_internal_norm[df_internal_norm["cycle"] == 1]["sample"]
df_internal_norm_mean_sd["sample"] = df_internal_norm[df_internal_norm["cycle"] == 1]["sample"]
#df_internal_norm_mean = df_internal_norm_mean[1:]
#df_internal_norm_mean = df_internal_norm_mean.drop(datetime.strptime("20150310 23:59", "%Y%m%d %H:%M"))
#df_internal_norm_mean_sd = df_internal_norm_mean_sd[1:]
df_internal_norm_mean_sd
#plotplot = df_internal_norm_mean[(df_internal_norm_mean["125"] < 0.00010)  & (df_internal_norm_mean["124/120"] > 0.17745)] 
#plot(plotplot["125"], plotplot["124/120"], "ro")

112/120                 114/120                 115/120  \
             mean            SE      mean            SE      mean   
counter                                                             
0        0.029976  1.869334e-07  0.020386  1.026412e-07  0.010365   
1        0.029814  1.869343e-07  0.020195  1.026377e-07  0.010365   

                        116/120                 117/120                \
                   SE      mean            SE      mean            SE   
counter                                                                 
0        1.475485e-07  0.446049  1.030892e-07  0.235314  1.607036e-07   
1        1.475488e-07  0.446001  1.030624e-07  0.235314  1.606992e-07   

            ...            118       119                  120            \
            ...             SE      mean        SE       mean        SE   
counter     ...                                                           
0           ...       0.000003  3.767052  0.000002  14.299983  0.000002   
1           ...       0.000003  3.767052  0.000002  14.299983  0.000002   

              122                124                 sample  
             mean        SE     mean        SE               
counter                                                      
0        2.031812  0.000002  2.53951  0.000003  SPEX 100ppb  
1        2.031812  0.000002  2.53951  0.000003  SPEX 100ppb  

[2 rows x 39 columns]

In [235]:
corr_values = df_internal_norm_mean
std_mean = corr_values[(corr_values.index == 1)].mean()
new = corr_values[corr_values.columns[corr_values.dtypes != 'object']]


epsilon =  ((new / std_mean)-1)*10000
epsilon[:1]

,112/120,114/120,115/120,116/120,117/120,118/120,119/120,122/120,124/120,112,114,115,116,117,118,119,120,122,124
counter,,,,,,,,,,,,,,,,,,,
0,54.528802,94.531014,-0.023817,1.080557,-0.014167,-0.00941,-0.004679,0.009257,0.018367,54.569122,94.559992,0,1.099536,0,0,0,0,0,0
